In [1]:
import sys
import os
import warnings
import time

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)
warnings.filterwarnings("ignore")

In [2]:
from src.database.db_connection import *
from src.evaluation.metrics import *
from src.utilities.step1_simplify_fds import *

In [3]:
engine = create_engine_for_db("primary")

In [4]:
query_file_name_expected_CQA = "../../../queries/FP/holoclean/hospital_truth.sql"
query_file_name_actual_CQA = "../../../queries/FP/holoclean/hospital_CQA.sql"
algorithm_name = ''
old_table_name = ''
new_table_name = ''

In [5]:
tids_query = text("""
    SELECT DISTINCT tid
    FROM prototype_fp.hospital_clean
    ORDER BY tid
""")

with engine.connect() as connection:
    tids = connection.execute(tids_query).scalars().all()

In [6]:
total_precision_CQA, total_recall_CQA,total_coverage_CQA, total_noise_CQA = 0.0, 0.0, 0.0, 0.0
for tid in tids:
    parameters = {"tid": tid}
    results_CQA,results_with_truth_set_CQA = calculate_expected_values(query_file_name_expected_CQA, query_file_name_actual_CQA, algorithm_name,
                                                                             engine,old_table_name,new_table_name,parameters)
    precision_CQA, recall_CQA,coverage_CQA, noise_CQA = summarize_results(results_CQA)
    total_precision_CQA += precision_CQA
    total_recall_CQA += recall_CQA
    total_coverage_CQA += coverage_CQA
    total_noise_CQA += noise_CQA

avg_precision_CQA = total_precision_CQA / len(tids)
avg_recall_CQA = total_recall_CQA / len(tids)
avg_coverage_CQA = total_coverage_CQA / len(tids)
avg_noise_CQA = total_noise_CQA / len(tids)

In [7]:
print(f"Average Precision: {avg_precision_CQA:.4f}")
print(f"Average Recall: {avg_recall_CQA:.4f}")
print(f"Average Coverage: {avg_coverage_CQA:.4f}")
print(f"Average Noise: {avg_noise_CQA:.4f}")

Average Precision: 1.0000
Average Recall: 0.0270
Average Coverage: 0.0270
Average Noise: 0.0000
